In [1]:
import pandas as pd
from datetime import date

In [2]:
from broker_limit import broker_limit
config_path="config.env"
broker_obj=broker_limit(config_path)
# broker_obj.ssh_host
conn=broker_obj.make_db_connection()

C:\Users\Lenovo\anaconda3\envs\Bobtail\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\Lenovo\anaconda3\envs\Bobtail\Lib\site-packages\paramiko\transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [3]:
# date_start='2025-07-01'
# date_end='2025-07-31'
debtor_id="177e2ce0-d431-4f83-87aa-3cdcf5ae313a" # rocky mountain
# debtor_id="f8477971-d55f-419b-88a8-a90687f2c1eb" # cowan
# debtor_id="f7498290-c18c-439d-a90c-b2a717b8b36b" # sage
# debtor_id="15932bdd-f6d8-402a-ab65-d1e742b81437" #TQL
# debtor_id='ROCKY MOUNTAIN EXPRESS CORP'
# debtor_id="f7498290-c18c-439d-a90c-b2a717b8b36b"
# debtor_id = urllib.parse.quote(debtor_id)
query = "select * from invoices i where debtor_id=%s and created_at>=current_date-455 and i.approved_date is not null"
cur = conn.cursor()
cur.execute(query, (debtor_id,))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
invoice_df=pd.DataFrame(results, columns=col_name)
invoice_df['approved_accounts_receivable_amount']=invoice_df['approved_accounts_receivable_amount']/100

query = "select * from debtors d where id=%s"
cur = conn.cursor()
cur.execute(query, (debtor_id,))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
debtors_df=pd.DataFrame(results, columns=col_name)
debtors_df['debtor_limit']=debtors_df['debtor_limit']/100
debtors_df['approved_total']=debtors_df['approved_total']/100

query = "select * from brokers b where debtor_id=%s"
cur = conn.cursor()
cur.execute(query, (debtor_id,))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
brokers_df=pd.DataFrame(results, columns=col_name)

query = "select * from clients c"
cur = conn.cursor()
cur.execute(query)
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
clients_df=pd.DataFrame(results, columns=col_name)

In [22]:
date_today=date.today()
date_last_year=date_today - pd.Timedelta(days=365)
# start_date=(date_last_year + pd.Timedelta(days=(8-date_last_year.isoweekday())%7) + pd.Timedelta(days=7))-pd.Timedelta(days=1) # for weekly
start_date=invoice_df['approved_date'].min().date() # for monthly
end_date=date_today

broker_level_df=broker_limit.generate_segment_level_data(start_date, end_date, debtors_df, brokers_df, invoice_df, step='monthly')
# broker_level_current=broker_limit.generate_segment_level_data(start_date=None, end_date=end_date, debtors_df=debtors_df, brokers_df=brokers_df, invoice_df=invoice_df, step='current')
# broker_level_df=broker_level[broker_level['dtp'].isna()==False]
# broker_level_df=pd.concat([broker_level, broker_level_current], ignore_index=True)
broker_level_df.head()

,snapshot_date,id,name,rating,approved_total,debtor_limit,mc,dot,longevity_in_days,invoice_approved,invoice_approved_dollars,invoice_paid,invoice_paid_dollars,dtp,open_invoices_till_date,open_invoices_in_point,no_of_clients,avg_ageing
0,2024-07-30,177e2ce0-d431-4f83-87aa-3cdcf5ae313a,ROCKY MOUNTAIN EXPRESS CORP,A,10100.0,77000.0,182548,2213127,1937,1.0,950.00,0.0,0.0,NaN,0.0,0.00,1.0,12.4
1,2024-08-29,177e2ce0-d431-4f83-87aa-3cdcf5ae313a,ROCKY MOUNTAIN EXPRESS CORP,A,10100.0,77000.0,182548,2213127,1937,2.0,4070.00,0.0,0.0,NaN,0.0,5020.00,2.0,12.4
2,2024-09-28,177e2ce0-d431-4f83-87aa-3cdcf5ae313a,ROCKY MOUNTAIN EXPRESS CORP,A,10100.0,77000.0,182548,2213127,1937,5.0,5000.00,3.0,5020.0,37.666667,0.0,5000.00,2.0,12.4
3,2024-10-28,177e2ce0-d431-4f83-87aa-3cdcf5ae313a,ROCKY MOUNTAIN EXPRESS CORP,A,10100.0,77000.0,182548,2213127,1937,18.0,18523.78,3.0,3000.0,34.000000,0.0,19523.78,4.0,12.4
4,2024-11-27,177e2ce0-d431-4f83-87aa-3cdcf5ae313a,ROCKY MOUNTAIN EXPRESS CORP,A,10100.0,77000.0,182548,2213127,1937,21.0,18750.00,18.0,15030.0,34.555556,0.0,24243.78,4.0,12.4


In [23]:
# broker_level_df=broker_level_df[broker_level_df['dtp'].isna()==False]
# broker_level_df

In [24]:
pivot_table, df_t, pivot_table_client_conc=broker_limit.generate_report(broker_level_df, broker_profile_report=True, cohort=['all', 6],payment_trend_count=6, payment_trend_step='default', debtors_df=debtors_df, brokers_df=brokers_df, invoice_df=invoice_df)

In [18]:
# df_t=broker_limit.payment_trend(broker_level_df, count=23, step='daily', debtors_df=debtors_df, brokers_df=brokers_df, invoice_df=invoice_df)

In [25]:
# pivot_table.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//tql_27_10_payment_trend.csv')

with pd.ExcelWriter('C://Users//Lenovo//Documents//Work//broker_profile_report//rocky_27_10.xlsx') as writer:
    pivot_table.to_excel(writer, sheet_name='Sheet1', index=True)
    df_t.to_excel(writer, sheet_name='Sheet2', index=True)
    pivot_table_client_conc.to_excel(writer, sheet_name='Sheet3', index=True)
    

In [ ]:
graph_df=df_t.T.reset_index()
graph_df

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df=graph_df

# Create one trace per KPI
fig = go.Figure([
    go.Scatter(x=df['snapshot_date'], y=df['invoice_approved'], mode='lines+markers', name='Approved invoices', yaxis='y2'),
    go.Scatter(x=df['snapshot_date'], y=df['invoice_paid'], mode='lines+markers', name='Paid Invoices', yaxis='y2'),
    go.Scatter(x=df['snapshot_date'], y=df['open_invoices_in_point'], mode='lines+markers', name='Open Invoices', yaxis='y1'),
    # go.Scatter(x=df['snapshot_date'], y=df['invoice_approved_dollars'], mode='lines+markers', name='Invoices Approved (dollars)', yaxis='y1'),
    # go.Scatter(x=df['snapshot_date'], y=df['invoice_paid_dollars'], mode='lines+markers', name='Invoices Paid (dollars)', yaxis='y1')
])

fig.update_layout(
    title="Broker Weekly Invoice Trend",
    xaxis_title="Date",
    yaxis_title="Open invoices amount",
    template="plotly_white",
    yaxis2=dict(
    title='Invoice count amount',
    overlaying='y',     # overlay on the same plotting area
    side='right'        # place on right
    ),
    legend=dict(x=1.1, y=1.1),
    height=500
)

fig.show()


In [65]:
# with pd.ExcelWriter('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_21_10.xlsx') as writer:
#     pivot_table.to_excel(writer, sheet_name='Sheet1', index=False)
#     df_t.to_excel(writer, sheet_name='Sheet2', index=False)
# pivot_table.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//sage_24_10.csv')
# df_t.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//tql_24_10_daily_payments_2.csv')
# pivot_table_client_conc.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//SAGE_23_10_client_conc.csv')

2025-10-24 20:30:13,597| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [ ]:
# pivot_table.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_21_10.csv')
# df_t.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_21_10_current_daily.csv')
# !pip install openpyxl

In [ ]:
# client concentration

In [ ]:
years_in_business=(date.today()-invoice_df.sort_values('created_at', ascending=True).iloc[0]['created_at'].date()).days / 365

debtor_level=invoice_df.groupby('debtor_id').agg(unique_clients=('client_id', 'nunique'), total_approved_invoices_dollars=('approved_accounts_receivable_amount', 'sum'), total_approved_invoices=('id', 'nunique')).reset_index()
debtor_level['invoice_amount_per_client']=debtor_level['total_approved_invoices_dollars'] / debtor_level['unique_clients']
debtor_level['years_in_business']=years_in_business

debtor_client_level=invoice_df.groupby(['debtor_id','client_id']).agg(client_approved_invoices=('approved_accounts_receivable_amount', 'sum')).reset_index().sort_values('client_approved_invoices', ascending=False)

abc=debtor_client_level.merge(debtor_level, on='debtor_id', how='left')
abc['client_invoices_cumsum']=abc['client_approved_invoices'].cumsum()
abc['share']=(abc['client_invoices_cumsum']/abc['total_approved_invoices_dollars']) *100
abc = abc.reset_index().rename(columns={'index': 'row_number'})
abc['row_number'] += 1
# abc[abc['share']>=25]['row_number'].min()
# abc['share']
abc

In [ ]:
invoice_df.groupby(['debtor_id','client_id']).agg(client_approved_invoices=('approved_accounts_receivable_amount', 'sum')).reset_index().sort_values('client_approved_invoices', ascending=False)

In [ ]:
invoice_df